___

# Energy A.I. Hackathon 2023 Workflow - Hackalopes 

#### Authors:   
<h4>  

**Richard Larson** - Energy and Earth Resources - *Jackson*  
**Karthik Menon** - Energy and Earth Resources - *Jackson*   
**Daniel Pang** - Petroleum and Geosystems Engineering - *Cockrell*    
**Benjamin Stormer** - Walker Department of Mechanical Engineering - *Cockrell* 

</h4> 

#### The University of Texas at Austin, Austin, Texas USA 
___

### Executive Summary 

We needed to predict whether or not 40 Electronic Submersible Pumps (ESP) will fail or not within 30 days. To create our predictions, we developed a data analytics and machine learning workflow in Python. We found that the GOR, ESP vibration data, and other factors related to fluid flow through the pump are most critical in assessing the status of an ESP. We recommend that feature engineering of these factors and ... be used as a measure to evaluate the lifespan of ESPs in the future.

___

### Workflow Goal

Our workflow should clearly provide the steps to select the optimal features for evaluating ESP lifespan, impute or remove any appropriate data, and develop a predictive learning model to categorize ESPs as expected to fail or not to fail in a period of 30 days.
___

### Workflow Steps 

Enumerated steps, very short and concise overview of your methods and workflow

1. **Data Analysis** - Separate data in failing pumps from operational pumps and understand feature completeness and correlation
2. **Feature Selection** - Leverage physical knowledge and domain expertise to determine most influential features, while imputing any data when possible, and removing any redundancies when necessary.
3. **Machine Learning Model \#1** - Create a minimum viable prototype using simple machine learning methods
4. **Machine Learning Model \#2** - Improve the minimum viable prototype by increasing complexity and changing feature selections


### Import Packages

In [23]:
import numpy as np                    # model arrays
import pandas as pd                   # DataFrames
import matplotlib.pyplot as plt       # building plots
import seaborn as sns                 # Plotting help
import os                             # accessing the operating system
from sklearn.impute import KNNImputer # k-nearest imputing
from sklearn.experimental import enable_iterative_imputer # required for MICE imputation
from sklearn.impute import IterativeImputer # MICE imputation

### Load Data

The following workflow applies the .csv files 'wellData.csv' and 'dailyData.csv'. 'wellData.csv' is a collection of static features related to a specific well. 'dailyData.csv' is a collection of time-series data particular to the operation of each pump. The datasets were made available to competitors as part of the UT Austin 2023 PGE Hackathon.

We will work with the following features:
* **DLS_Critical** - ...
* **GOR** - ...
* **ESP Data - Vibration X** - ...
* **Pump_Power** - ...

In [24]:
# Relative paths used since assumption is that user is in root directory of forked Git repo.
well_data = pd.read_csv("../Hackalopes/wellData.csv")      # load the well data in
daily_data = pd.read_csv("../Hackalopes/dailyData.csv")    # load the daily data in
solution_data = pd.read_csv("../Hackalopes/solution.csv")  # load the solution data in

### Functions

The following functions will be used in the workflow.

In [25]:
# Plots a correlation matrix as a heat map
def plot_corr(dataframe,size=10):                        
    corr = dataframe.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    im = ax.matshow(corr,vmin = -1.0, vmax = 1.0)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);
    plt.colorbar(im, orientation = 'vertical')
    plt.title('Correlation Matrix')


# Replaces the NaN values in a dataset
def count_and_replace_nan(dataset, feature, replacement_value):
    num_nan = sum(dataset[feature].isna())
    print(f"Number of NaN values in {feature} is {num_nan} out of {len(dataset)} values")
    if replacement_value == "bf":
        new = dataset[feature].fillna(method="bfill", inplace=True)
        print(f"NaN values have been replaced by back-filling")
    elif replacement_value == "ff":
        dataset[feature].fillna(method="ffill", inplace=True)
        print(f"NaN values have been replaced by forward-filling")
    else:
        dataset[feature].fillna(replacement_value, inplace=True)
        print(f"NaN values have been replaced with {replacement_value}")
    return dataset


# Compute the completeness metrics for a given dataset
def completeness(df, cutoff=0.75):
    df_temp = df.copy(deep=True)
    df_bool = df_temp.isnull()
    features = list(df_bool.columns)
    percent_missing = []
    past_cutoff = []
    for feature in features:
        num_missing = df_bool[feature].sum()
        proportion_missing = df_bool[feature].sum() / len(df_bool)
        percent_missing.append(proportion_missing)
        if proportion_missing >= cutoff:
            past_cutoff.append(feature)
    print(f'{past_cutoff} were missing at least {cutoff*100}% of their data')


# Calculate the (spearman) correlation between all features in a dataframe.
def correlation(df, cutoff=0.75):
    corr = df.corr(method='spearman')
    correlated = []
    inversely_correlated = []
    for feature in corr:
        for feature2 in corr:
            correlation = corr[feature][feature2]
            if str(feature) == str(feature2):
                pass
            elif correlation >= cutoff:
                correlated.append((feature, feature2))
            elif correlation <= -cutoff:
                inversely_correlated.append((feature, feature2))
    # print(f'Items listed here: {correlated} are correlated by a coefficient of at least {cutoff}')
    # print('-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
    # print(f'Items listed here: {inversely_correlated} are inversely correlated by a coefficient of at greatest -{cutoff}')


    # Function to separate a feature by time intervals
def separate_intervals(dataset, feature):
    grouped_data=dataset.groupby(["Well_ID","AL_Key"])
    array_life=np.array(grouped_data.size())
    mega_matrix=np.array(grouped_data)
    test_array=np.array(dataset) # Get the Well_ID
    list_sort=[]
    list_fail=[]
    for i in range (0,len(mega_matrix)):
        flag=0
        for k in range(0,len(test_array)):        
            if mega_matrix[i][0][0] == test_array[k][0] and mega_matrix[i][0][1] == test_array[k][1]:
                list_sort.append(i)
                flag=1            
        if flag==0:
            list_fail.append(i)
    mega_s=mega_matrix[list_sort]
    mega_s_life=array_life[list_sort]
    
    mega_f=mega_matrix[list_fail]
    mega_f_life=array_life[list_fail]

    old_folks=[]
    for i in range (len(mega_f)):
        overall_time=mega_f_life[i]
        senior=overall_time-int((overall_time*0.1).round(0))
        alpha=mega_f[i][1][senior:overall_time][feature].diff().mean()
        old_folks.append(alpha)

    army=[]
    for i in range (len(mega_f)):
        overall_time=mega_f_life[i]
        start=overall_time-int((overall_time*0.6).round(0))
        end=overall_time-int((overall_time*0.4).round(0))
        soldier=mega_f[i][1][start:end][feature].diff().mean()
        army.append(soldier)
        
    return([old_folks, army])

### Aggregate and Separate Data Appropriately

The following steps combine the input csv files into DataFrames that are more useful to manage

In [26]:
# Mark data we will be predicting with a 0 and failed pumps with a 1
solution_data = solution_data.rename(columns={"Fail in 30 days": "Failed"}) 
solution_data["Failed"] = 0 # Initialize solutions with 0

In [27]:
# Combine all data into one dataframe
combined_data = pd.merge(pd.merge(well_data, daily_data, on=["Well_ID", "AL_Key"], how="left"), 
                         solution_data, on=["Well_ID", "AL_Key"], how="left")
combined_data["Failed"] = combined_data["Failed"].replace(np.nan, 1)
combined_data = combined_data.drop(columns=["Unnamed: 0"])

In [28]:
# Backfill NaN values in OIL, GAS, and Water columns
combined_data["OIL"].fillna(method="bfill", inplace=True)
combined_data["GAS"].fillna(method="bfill", inplace=True)
combined_data["WATER"].fillna(method="bfill", inplace=True)

In [29]:
# Insert new ratios and features in dataframe
combined_data.insert(15, "GOR", combined_data["GAS"] / combined_data["OIL"])
combined_data.insert(16, "GOF", combined_data["GAS"] / (combined_data["OIL"] + combined_data["WATER"]))
combined_data.insert(17, "GOR_Slope", combined_data["GOR"].diff())

In [30]:
# Create a dataframe for both the pumps still operational and the pumps that have failed
solution_data = combined_data[combined_data["Failed"] == 0] # Still operational
failed_data = combined_data[combined_data["Failed"] == 1] # Failed

### Basic Data Checking and Visualization

In [31]:
correlation(combined_data) # Compute the correlation statistics for the combined dataframe

In [32]:
completeness(solution_data) # Compute the data completeness statistics for the operational pumps

['ESP_Motor_Frequency_Rating'] were missing at least 75.0% of their data


### Compute Statistics for the Minimum Viable Prototype

In [33]:
# Remove inessential data for a preliminary prototype. Essential data 
# determined from physical knowldege from domain experts and 
# correlation/completeness studies.
min_viable = combined_data.drop(columns=['Artificial_Lift_Type',
       'AL_Bottom_Depth', 'ESP_Pump_Stages',
       'ESP_Motor_Frequency_Rating', 'ESP_Motor_Current_Rating',
       'ESP_Motor_Voltage_Rating', 'ESP_Motor_Power_Rating',
       'DLS_at_Set_Depth', 'OIL', 'GAS', 'WATER', 'ARTIFICIAL_LIFT',
       'DOWN_TIME_HOURS', 'ESP Data - Drive Current',
       'ESP Data - Drive Voltage', 'ESP Data - Intake Pressure',
       'ESP Data - Motor Temperature Shutdown Setpoint',
       'ESP Data - Motor Winding Temperature', 'ESP Data - Output Frequency',
       'Startup_Count', 'Oil_Intake', 'Water_Intake',
       'Gas_Intake', 'Liquid_Intake', 'Gas_Saturation_at_Intake',
       'Gas_Separator_Efficiency', 'Gas_through_Annulus_Intake',
       'Gas_through_ESP_Intake', 'Gas_through_Annulus', 'Gas_through_ESP',
       'Pb_ESP', 'Discharge_Pressure', 'ESP_Fluid',
       'Gas_Saturation_at_Discharge', 'Pump_Delta_Pressure',
       'Pump_Average_Pressure', 'Gas_Saturation_in_Pump',
       'Drive_Power', 'Power_Ratio', 'Power_Difference', 'ESP_Temperature',
       'Lower_Limit', 'Failed'])

### Compute Statistics for the Improved Model
Two methods of imputation are used and tested separately

In [34]:
# Remove inessential data for an improved prototype. This is slightly less
# strict in removing features then the minimum viable prototype
improved = combined_data.drop(columns=['Artificial_Lift_Type',
       'AL_Bottom_Depth', 'ESP_Pump_Stages',
       'ESP_Motor_Frequency_Rating', 'ESP_Motor_Current_Rating',
       'ESP_Motor_Voltage_Rating', 'ESP_Motor_Power_Rating',
       'DLS_at_Set_Depth', 'GAS', 'WATER', 'ARTIFICIAL_LIFT',
       'DOWN_TIME_HOURS', 'ESP Data - Drive Current',
       'ESP Data - Drive Voltage', 'ESP Data - Intake Pressure',
       'ESP Data - Motor Temperature Shutdown Setpoint',
       'ESP Data - Motor Winding Temperature', 'ESP Data - Output Frequency',
       'Startup_Count', 'Oil_Intake', 'Water_Intake',
       'Gas_Intake', 'Liquid_Intake', 'Gas_Saturation_at_Intake',
       'Gas_Separator_Efficiency', 'Gas_through_Annulus_Intake',
       'Gas_through_ESP_Intake', 'Gas_through_Annulus', 'Gas_through_ESP',
       'Discharge_Pressure', 'ESP_Fluid',
       'Gas_Saturation_at_Discharge', 'Pump_Delta_Pressure',
       'Pump_Average_Pressure', 'Gas_Saturation_in_Pump',
       'Drive_Power', 'Power_Difference', 'ESP_Temperature',
       'Lower_Limit', 'Failed'])

In [35]:
# Code modified from https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/SubsurfaceDataAnalytics_Feature_Imputation.ipynb
# Authored by Dr. Michael Pyrcz

# Impute the data using k-nearest neighbors
df_knn = improved.copy(deep=True) # make a deep copy of the DataFrame
df_knn.drop(columns=["Well_ID", "AL_Key"], inplace=True)
df_knn.replace([np.inf, -np.inf], np.nan, inplace=True) # Replace infinite values
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
df_knn.iloc[:,:] = knn_imputer.fit_transform(df_knn)

In [36]:
# Code modified from https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/SubsurfaceDataAnalytics_Feature_Imputation.ipynb
# Authored by Dr. Michael Pyrcz

# Impute the data using chained equations
df_mice = improved.copy(deep=True)  # make a deep copy of the DataFrame
df_mice.drop(columns=["Well_ID", "AL_Key"], inplace=True)
df_mice.replace([np.inf, -np.inf], np.nan, inplace=True) # Replace infinite values
mice_imputer = IterativeImputer()
df_mice.iloc[:,:] = mice_imputer.fit_transform(df_mice)